## Table of Contents
- [0 - Overview](#0)
    - [0.1 - Edit Distance](#0-1)
- [1 - Data Preprocessing](#1)
    - [process_data](#ex-1)
    - [get_count](#ex-2)
    - [get_probs](#ex-3)
- [2 - String Manipulations](#2)
    - [delete_letter](#ex-4)
    - [switch_letter](#ex-5)
    - [replace_letter](#ex-6)
    - [insert_letter](#ex-7)
- [3 - Combining the Edits](#3)
    - [3.1 - Edit One Letter](#3-1)
        - [edit_one_letter](#ex-8)
    - [3.2 - Edit Two Letters](#3-2)
        - [edit_two_letters](#ex-9)
    - [3.3 - Suggest Spelling Suggestions](#3-3)
        - [get_corrections](#ex-10)
- [4 - Minimum Edit Distance](#4)
    - [4.1 - Dynamic Programming](#4-1)
        - [min_edit_distance](#ex-11)


<a name='0'></a>
## 0 - Overview

Autocorrect is a feature commonly used on cell phones and computers. This project delves into the mechanisms behind autocorrect. While the model implemented here is not identical to the one on your phone, it is still quite effective.

Through this project, one will learn how to:

- Obtain a word count from a corpus
- Determine word probability within the corpus
- Manipulate and filter strings
- Implement minimum edit distance to compare strings and find the optimal editing path
- Understand the principles of dynamic programming

Such systems are prevalent in various applications. For instance, if someone types **"I am lerningg"**, it is highly likely they intended to write **"learning"**, as illustrated in **Figure 1**.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='images/auto-correct.png' alt="alternate text" width="width" height="height" style="width:300px;height:250px;" /> Figure 1 </div>

<a name='0-1'></a>
### 0.1 - Edit Distance

This project involves implementing models that correct words that are one or two edit distances away. Two words are said to be n edit distances apart when n edits are required to transform one word into the other.

An edit can consist of one of the following actions:

- Deletion (removing a letter): ‘hat’ => ‘at, ha, ht’
- Switching (swapping two adjacent letters): ‘eta’ => ‘eat, tea,...’
- Replacement (changing one letter to another): ‘jat’ => ‘hat, rat, cat, mat, ...’
- Insertion (adding a letter): ‘te’ => ‘the, ten, ate, ...’

These four methods will be used to implement an auto-correct feature. To achieve this, probabilities must be computed to determine the likelihood that a given word is correct.

The auto-correct model being implemented was originally created by [Peter Norvig](https://en.wikipedia.org/wiki/Peter_Norvig) in 2007. His [original article](https://norvig.com/spell-correct.html) can serve as a useful reference.

The objective of the spell check model is to compute the following probability:

$$P(c|w) = \frac{P(w|c)\times P(c)}{P(w)} \tag{Eqn-1}$$

This equation is derived from [Bayes' Rule](https://en.wikipedia.org/wiki/Bayes%27_theorem).
- Equation 1 states that the probability of a word being correct \(P(c|w)\) is equal to the probability of having a certain word \(w\), given that it is correct \(P(w|c)\), multiplied by the probability of the word being correct in general \(P(c)\), divided by the probability of the word \(w\) appearing in general \(P(w)\).
- To compute this equation, a dataset will be imported, and the necessary probabilities will be created using that dataset.

<a name='1'></a>
## 1 - Data Preprocessing 

In [53]:
import re
from collections import Counter
import numpy as np
import pandas as pd

import w1_unittest

- As in any other machine learning task, the first thing we have to do is process your data set. 
- The first task is to read in a file called 'shakespeare.txt'.

<a name='ex-1'></a>
### process_data

Function `process_data`: 

- Reads in a corpus (text file)

- Changes everything to lowercase

- Returns a list of words. 

In [54]:
import re

def process_data(file_name):

    words = [] # return this variable correctly

    ### START CODE HERE ### 
    with open(file_name) as f:
        lines = f.read()
        
    lines = lines.lower()
    words = re.findall('\w+', lines)
    
    return words


Note, in the following cell, 'words' is converted to a python `set`. This eliminates any duplicate entries.

In [55]:
#DO NOT MODIFY THIS CELL
word_l = process_data('./data/shakespeare.txt')
vocab = set(word_l)  # this will be your new vocabulary
print(f"The first ten words in the text are: \n{word_l[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['o', 'for', 'a', 'muse', 'of', 'fire', 'that', 'would', 'ascend', 'the']
There are 6116 unique words in the vocabulary.


In [56]:
# Test your function
w1_unittest.test_process_data(process_data)

 All tests passed


<a name='ex-2'></a>

### get_count

`get_count` function returns a dictionary
- The dictionary's keys are words
- The value for each word is the number of times that word appears in the corpus. 

For example, given the following sentence: **"I am happy because I am learning"**, your dictionary should return the following: 
<table style="width:20%">

  <tr>
    <td> <b>Key </b>  </td>
    <td> <b>Value </b> </td> 


  </tr>
  <tr>
    <td> I  </td>
    <td> 2</td> 
 
  </tr>
   
  <tr>
    <td>am</td>
    <td>2</td> 
  </tr>

  <tr>
    <td>happy</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>because</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>learning</td>
    <td>1</td> 
  </tr>
</table>


In [57]:
def get_count(word_l):
    
    word_count_dict = {}
    
    for word in word_l:
        if word in word_count_dict:
            word_count_dict[word] += 1
        else:
            word_count_dict[word] = 1
            
    return word_count_dict

In [58]:
#DO NOT MODIFY THIS CELL
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'thee' is {word_count_dict.get('thee',0)}")

There are 6116 key values pairs
The count for the word 'thee' is 240


In [59]:
# Test your function
w1_unittest.test_get_count(get_count, word_l)

 All tests passed


<a name='ex-3'></a>
### get_probs
Given the dictionary of word counts, computes the probability that each word will appear if randomly selected from the corpus of words.

$$P(w_i) = \frac{C(w_i)}{M} \tag{Eqn-2}$$
where 

$C(w_i)$ is the total number of times $w_i$ appears in the corpus.

$M$ is the total number of words in the corpus.

For example, the probability of the word 'am' in the sentence **'I am happy because I am learning'** is:

$$P(am) = \frac{C(w_i)}{M} = \frac {2}{7} \tag{Eqn-3}.$$

In [60]:
def get_probs(word_count_dict):

    probs = {}
    
    # gets the total count of words for all words in the dictionary
    total_count = sum(word_count_dict.values())
    
    # calculates the probability for each word
    for word, count in word_count_dict.items():
        probs[word] = count / total_count
    
    return probs


In [61]:
#DO NOT MODIFY THIS CELL
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('thee') is {probs['thee']:.4f}")

Length of probs is 6116
P('thee') is 0.0045


In [62]:
# Test your function
w1_unittest.test_get_probs(get_probs, word_count_dict)

 All tests passed


<a name='2'></a>
## 2 - String Manipulations

Now that we have computed $P(w_i)$ for all the words in the corpus, we will write a few functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words. In this section, we will implement four functions: 

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 


<a name='ex-4'></a>
### delete_letter

`delete_letter()` function , given a word, returns a list of strings with one character deleted. 

For example, given the word **nice**, it would return the set: {'ice', 'nce', 'nic', 'nie'}. 

**Step 1:** Create a list of 'splits'. This is all the ways we can split a word into Left and Right: For example,   
'nice is split into : `[('', 'nice'), ('n', 'ice'), ('ni', 'ce'), ('nic', 'e'), ('nice', '')]`
This is common to all four functions (delete, replace, switch, insert).


**Step 2:** This is specific to `delete_letter`. Here, we are generating all words that result from deleting one character.  
This can be done in a single line with a list comprehension. We can make use of this type of syntax:  
`[f(a,b) for a, b in splits if condition]`  

For our 'nice' example you get: 
['ice', 'nce', 'nie', 'nic']

In [63]:
def delete_letter(word, verbose=False):
    
    delete_l = []
    split_l = []
    
    # Generate all split pairs
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    # Generate all possible words by deleting one character
    delete_l = [L + R[1:] for L, R in split_l if R]
    
    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return  delete_l


In [64]:
delete_word_l = delete_letter(word="cans",
                        verbose=True)

input word cans, 
split_l = [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's'), ('cans', '')], 
delete_l = ['ans', 'cns', 'cas', 'can']


In [65]:
# test # 2
print(f"Number of outputs of delete_letter('at') is {len(delete_letter('at'))}")

Number of outputs of delete_letter('at') is 2


In [66]:
# Test your function
w1_unittest.test_delete_letter(delete_letter)

 All tests passed


<a name='ex-5'></a>
### switch_letter

**switch_letter()**: function switches two letters in a word. It takes in a word and returns a list of all the possible switches of two letters **that are adjacent to each other**. 
- For example, given the word 'eta', it returns {'eat', 'tea'}, but does not return 'ate'.

**Step 1:** is the same as in delete_letter()  
**Step 2:** A list comprehension or for loop which forms strings by swapping adjacent letters. This is of the form:  
`[f(L,R) for L, R in splits if condition]`  where 'condition' will test the length of R in a given iteration. See below.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='images/Switches1.PNG' alt="alternate text" width="width" height="height" style="width:600px;height:200px;"/> Figure 5 </div>      

In [67]:
def switch_letter(word, verbose=False):
    
    switch_l = []
    split_l = []
    
   
    # Generate all split pairs
    split_l = [(word[:i], word[i:]) for i in range(len(word) - 1)]
    
    # Generate all possible words by switching adjacent characters
    switch_l = [L + R[1] + R[0] + R[2:] for L, R in split_l if len(R) > 1]
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 
    
    return switch_l


In [68]:
switch_word_l = switch_letter(word="eta",
                         verbose=True)

Input word = eta 
split_l = [('', 'eta'), ('e', 'ta')] 
switch_l = ['tea', 'eat']


In [69]:
# test # 2
print(f"Number of outputs of switch_letter('at') is {len(switch_letter('at'))}")

Number of outputs of switch_letter('at') is 1


In [70]:
# Test your function
w1_unittest.test_switch_letter(switch_letter)

 All tests passed


<a name='ex-6'></a>
### replace_letter

**replace_letter()**: function that takes in a word and returns a list of strings with one **replaced letter** from the original word. 

**Step 1:** is the same as in `delete_letter()`

**Step 2:** A list comprehension or for loop which form strings by replacing letters. This can be of the form:  
`[f(a,b,c) for a, b in splits if condition for c in string]`   Note the use of the second for loop.  
It is expected in this routine that one or more of the replacements will include the original word. For example, replacing the first letter of 'ear' with 'e' will return 'ear'.

**Step 3:** Remove the original input letter from the output.

In [71]:
def replace_letter(word, verbose=False):
        
    letters = 'abcdefghijklmnopqrstuvwxyz'
    
    replace_l = []
    split_l = []
    
    
    # Generate all split pairs
    split_l = [(word[:i], word[i:]) for i in range(len(word))]
    
    # Generate all possible words by replacing one character
    replace_l = [L + c + R[1:] for L, R in split_l if R for c in letters if c != R[0]]
    
    
    # Sort the list, for easier viewing
    replace_l = sorted(replace_l)
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l


In [72]:
replace_l = replace_letter(word='can',
                              verbose=True)

Input word = can 
split_l = [('', 'can'), ('c', 'an'), ('ca', 'n')] 
replace_l ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [73]:
# test # 2
print(f"Number of outputs of replace_letter('at') is {len(replace_letter('at'))}")

Number of outputs of replace_letter('at') is 50


In [74]:
# Test your function
w1_unittest.test_replace_letter(replace_letter)

 All tests passed


<a name='ex-7'></a>
### insert_letter

**insert_letter()**: function that takes in a word and returns a list with a letter inserted at every offset.

**Step 1:** is the same as in `delete_letter()`

**Step 2:** This can be a list comprehension of the form:  
`[f(a,b,c) for a, b in splits if condition for c in string]`   

In [75]:
def insert_letter(word, verbose=False):

    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
    # Generate all split pairs
    split_l = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    # Generate all possible words by inserting one new letter
    insert_l = [L + c + R for L, R in split_l for c in letters]
        
    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l


In [76]:
insert_l = insert_letter('at', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word at 
split_l = [('', 'at'), ('a', 't'), ('at', '')] 
insert_l = ['aat', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat', 'aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'att', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz']
Number of strings output by insert_letter('at') is 78


In [77]:
# Test your function
w1_unittest.test_insert_letter(insert_letter)

 All tests passed


<a name='3'></a>
## 3 - Combining the Edits

Now that we have implemented the string manipulations, we will create two functions that, given a string, will return all the possible single and double edits on that string. These will be `edit_one_letter()` and `edit_two_letters()`.

<a name='3-1'></a>
### 3.1 - Edit One Letter

<a name='ex-8'></a>
### edit_one_letter

`edit_one_letter` function to get all the possible edits that are one edit away from a word. The edits consist of the replace, insert, delete, and optionally the switch operation. We should use the previous functions we have already implemented to complete this function. The 'switch' function  is a less common edit function, so its use will be selected by an "allow_switches" input argument.

Note that those functions return *lists* while this function should return a *python set*. Utilizing a set eliminates any duplicate entries.

In [78]:
def edit_one_letter(word, allow_switches=True):
    """
    Input:
        word: the string/word for which we will generate all possible words that are one edit away.
        allow_switches: boolean indicating if adjacent letter switching is allowed.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set and not a list.
    """
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    edit_one_set = set()

    # Deletion
    for i in range(len(word)):
        edit_one_set.add(word[:i] + word[i+1:])
    
    # Insertion
    for i in range(len(word) + 1):
        for letter in letters:
            edit_one_set.add(word[:i] + letter + word[i:])
    
    # Substitution
    for i in range(len(word)):
        for letter in letters:
            if letter != word[i]:  # To avoid generating the same word
                edit_one_set.add(word[:i] + letter + word[i+1:])
    
    # Switch (if allowed)
    if allow_switches:
        for i in range(len(word) - 1):
            # Swap only if the letters are different to avoid generating the same word
            if word[i] != word[i+1]:
                edit_one_set.add(word[:i] + word[i+1] + word[i] + word[i+2:])
    
    return edit_one_set

In [79]:
tmp_word = "at"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word at 
edit_one_l 
['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']

The type of the returned object should be a set <class 'set'>
Number of outputs from edit_one_letter('at') is 129


In [80]:
# Test your function
w1_unittest.test_edit_one_letter(edit_one_letter)

 All tests passed


<a name='3-2'></a>
### 3.2 - Edit Two Letters

<a name='ex-9'></a>
### edit_two_letters

Now we can generalize this to implement to get two edits on a word. To do so, you would have to get all the possible edits on a single word and then for each modified word, you would have to modify it again. 

`edit_two_letters` function that returns a set of words that are two edits away. Note that creating additional edits based on the `edit_one_letter` function may 'restore' some one_edits to zero or one edits. That is allowed here. This is accounted for in get_corrections.

In [81]:
def edit_two_letters(word, allow_switches=True):
    """
    Input:
        word: the input string/word 
        allow_switches: boolean indicating if adjacent letter switching is allowed.
    Output:
        edit_two_set: a set of strings with all possible two edits
    """
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    edit_two_set = set()

    # First edit: Deletion, Insertion, Substitution, Switch (if allowed)
    first_edit_set = set()

    for i in range(len(word)):
        # Deletion
        first_edit_set.add(word[:i] + word[i+1:])
        
        # Substitution
        for letter in letters:
            if letter != word[i]:  # To avoid generating the same word
                first_edit_set.add(word[:i] + letter + word[i+1:])
        
        # Switch (if allowed)
        if allow_switches and i < len(word) - 1:
            # Swap only if the letters are different to avoid generating the same word
            if word[i] != word[i+1]:
                first_edit_set.add(word[:i] + word[i+1] + word[i] + word[i+2:])
    
    for i in range(len(word) + 1):
        # Insertion
        for letter in letters:
            first_edit_set.add(word[:i] + letter + word[i:])

    # Second edit: Apply all possible single edits to each word in the first edit set
    for edit in first_edit_set:
        for i in range(len(edit)):
            # Deletion
            edit_two_set.add(edit[:i] + edit[i+1:])
            
            # Substitution
            for letter in letters:
                if letter != edit[i]:  # To avoid generating the same word
                    edit_two_set.add(edit[:i] + letter + edit[i+1:])
            
            # Switch (if allowed)
            if allow_switches and i < len(edit) - 1:
                # Swap only if the letters are different to avoid generating the same word
                if edit[i] != edit[i+1]:
                    edit_two_set.add(edit[:i] + edit[i+1] + edit[i] + edit[i+2:])
        
        for i in range(len(edit) + 1):
            # Insertion
            for letter in letters:
                edit_two_set.add(edit[:i] + letter + edit[i:])

    return edit_two_set

# Test the function
print(edit_two_letters("word"))


{'weorcd', 'dortd', 'kordv', 'wordkp', 'wzrdg', 'wkrt', 'wordwm', 'wqordi', 'wfokrd', 'bwoord', 'cokrd', 'wtd', 'wworfd', 'worrzd', 'wozrdh', 'zuword', 'aorz', 'wovrm', 'wojvd', 'voqd', 'mwoqd', 'nordb', 'whofd', 'whrdg', 'zwuord', 'wkrdf', 'hwjrd', 'zwors', 'worddx', 'wozdj', 'rwkord', 'wjeord', 'efrd', 'owvd', 'worgrd', 'zorfd', 'wojn', 'hyrd', 'wdorqd', 'puword', 'wsoxrd', 'vwford', 'wobdq', 'uorb', 'worurd', 'cwcrd', 'woyard', 'ywordo', 'worqp', 'wromrd', 'wborm', 'wiorj', 'aorv', 'korvd', 'hzord', 'bwcrd', 'sorw', 'wygord', 'aworhd', 'wwrdu', 'wtordh', 'woldg', 'wnors', 'kwzord', 'wzrpd', 'lwaord', 'worodf', 'voord', 'wnoerd', 'woorjd', 'ifrd', 'worwc', 'fhord', 'wkordn', 'wobw', 'kwomrd', 'iwxord', 'wodur', 'wouard', 'woujrd', 'qwordy', 'pwdrd', 'wwld', 'whofrd', 'woydf', 'pwojd', 'weorwd', 'woxk', 'wborn', 'wdword', 'wcxd', 'wurzd', 'wlrsd', 'woraad', 'wvdord', 'whordf', 'wohd', 'wortdv', 'qwofd', 'zwoord', 'mokrd', 'wvrd', 'wzcd', 'uozd', 'bofd', 'wira', 'weqd', 'yorid', 'wdord

In [82]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'at' is 7154


In [83]:
# Test your function
w1_unittest.test_edit_two_letters(edit_two_letters)

 All tests passed


<a name='3-3'></a>
### 3.3 - Suggest Spelling Suggestions

The `edit_two_letters` function is used to generate a set of all possible two-letter edits for a given word. These generated strings are then used to find the most probable word that was intended, providing a typing suggestion.

<a name='ex-10'></a>
### get_corrections
The function `get_corrections` returns a list of zero to n possible suggestion tuples in the form of (word, probability_of_word).

**Step 1:** Generate suggestions for a given word using the developed edit functions. The 'suggestion algorithm' should follow this logic:
* If the word is in the vocabulary, suggest the word.
* If there are suggestions from `edit_one_letter` that are in the vocabulary, use those.
* If there are suggestions from `edit_two_letters` that are in the vocabulary, use those.
* If none of the above, suggest the input word.
* The idea is that words generated from fewer edits are more likely than words with more edits.

Note:
- Edits of two letters may 'restore' strings to either zero or one edit. This algorithm accounts for this by preferentially selecting lower distance edits first.

The logical `or` could be used to implement the suggestion algorithm very compactly. Alternately, if/elif/else constructs could be used.
 
**Step 2**: Create a 'best_words' dictionary where the 'key' is a suggestion and the 'value' is the probability of that word in your vocabulary. If the word is not in the vocabulary, assign it a probability of 0.

**Step 3**: Select the n best suggestions. There may be fewer than n.

In [84]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    

    res = []
    if word in probs:
        res += [(word, probs[word])]
    res += sorted([(w, probs[w]) for w in edit_one_letter(word) if w in probs], key = lambda x : -x[1])
    res += sorted([(w, probs[w]) for w in edit_two_letters(word) if w in probs], key = lambda x : -x[1])
    suggestions = set([x[0] for x in res[:n]])
    n_best = [x for x in res[:n]]

    
    if verbose: print("suggestions = ", suggestions)

    return n_best

In [85]:
# Test your implementation - feel free to try other words in my word
my_word = 'dys' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True) # keep verbose=True
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

# CODE REVIEW COMMENT: using "tmp_corrections" insteads of "cors". "cors" is not defined
print(f"data type of corrections {type(tmp_corrections)}")

suggestions =  {'dye', 'days'}
word 0: days, probability 0.000410
word 1: dye, probability 0.000019
data type of corrections <class 'list'>


In [86]:
# Test your function
w1_unittest.test_get_corrections(get_corrections, probs, vocab)

 All tests passed


<a name='4'></a>
## 4 - Minimum Edit Distance

To evaluate the similarity between two strings, such as 'waht' and 'what,' and to efficiently determine the shortest path to transform 'waht' into 'what,' a dynamic programming system is used. This system calculates the minimum number of edits required to convert one string into another.

<a name='4-1'></a>
### 4.1 - Dynamic Programming

Dynamic Programming breaks a problem down into subproblems which can be combined to form the final solution. Here, given a string source[0..i] and a string target[0..j], we will compute all the combinations of substrings[i, j] and calculate their edit distance. To do this efficiently, we will use a table to maintain the previously computed substrings and use those to calculate larger substrings.

You have to create a matrix and update each element in the matrix as follows:  

$$\text{Initialization}$$

\begin{align}
D[0,0] &= 0 \\
D[i,0] &= D[i-1,0] + del\_cost(source[i]) \tag{4}\\
D[0,j] &= D[0,j-1] + ins\_cost(target[j]) \\
\end{align}

So converting the source word **play** to the target word **stay**, using an input cost of one, a delete cost of 1, and replace cost of 2 would give you the following table:
<table style="width:20%">

  <tr>
    <td> <b> </b>  </td>
    <td> <b># </b>  </td>
    <td> <b>s </b>  </td>
    <td> <b>t </b> </td> 
    <td> <b>a </b> </td> 
    <td> <b>y </b> </td> 
  </tr>
   <tr>
    <td> <b>  #  </b></td>
    <td> 0</td> 
    <td> 1</td> 
    <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
 
  </tr>
  <tr>
    <td> <b>  p  </b></td>
    <td> 1</td> 
 <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
   <td> 5</td>
  </tr>
   
  <tr>
    <td> <b> l </b></td>
    <td>2</td> 
    <td>3</td> 
    <td>4</td> 
    <td>5</td> 
    <td>6</td>
  </tr>

  <tr>
    <td> <b> a </b></td>
    <td>3</td> 
     <td>4</td> 
     <td>5</td> 
     <td>4</td>
     <td>5</td> 
  </tr>
  
   <tr>
    <td> <b> y </b></td>
    <td>4</td> 
      <td>5</td> 
     <td>6</td> 
     <td>5</td>
     <td>4</td> 
  </tr>
  

</table>



The operations used in this algorithm are 'insert', 'delete', and 'replace'. These correspond to the functions that you defined earlier: insert_letter(), delete_letter() and replace_letter(). switch_letter() is not used here.

The diagram below describes how to initialize the table. Each entry in D[i,j] represents the minimum cost of converting string source[0:i] to string target[0:j]. The first column is initialized to represent the cumulative cost of deleting the source characters to convert string "EER" to "". The first row is initialized to represent the cumulative cost of inserting the target characters to convert from "" to "NEAR".

<div style="width:image width px; font-size:100%; text-align:center;"><img src='images/EditDistInit4.PNG' alt="alternate text" width="width" height="height" style="width:1000px;height:400px;"/> Figure 6 Initializing Distance Matrix</div>     

Filling in the remainder of the table utilizes the 'Per Cell Operations' in the equation (5) above. Note, the diagram below includes in the table some of the 3 sub-calculations shown in light grey. Only 'min' of those operations is stored in the table in the `min_edit_distance()` function.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='images/EditDistFill2.PNG' alt="alternate text" width="width" height="height" style="width:800px;height:400px;"/> Figure 7 Filling Distance Matrix</div>     

Note that the formula for $D[i,j]$ shown in the image is equivalent to:

\begin{align}
 \\
D[i,j] =min
\begin{cases}
D[i-1,j] + del\_cost\\
D[i,j-1] + ins\_cost\\
D[i-1,j-1] + \left\{\begin{matrix}
rep\_cost; & if src[i]\neq tar[j]\\
0 ; & if src[i]=tar[j]
\end{matrix}\right.
\end{cases}
\tag{5}
\end{align}

The variable `sub_cost` (for substitution cost) is the same as `rep_cost`; replacement cost.  We will stick with the term "replace" whenever possible.

Below are some examples of cells where replacement is used. This also shows the minimum path from the lower right final position where "EER" has been replaced by "NEAR" back to the start. This provides a starting point for the optional 'backtrace' algorithm below.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='images/EditDistExample1.PNG' alt="alternate text" width="width" height="height" style="width:1200px;height:400px;"/> Figure 8 Examples Distance Matrix</div>    

<a name='ex-11'></a>
### min_edit_distance

Again, the word "substitution" appears in the figure, but think of this as "replacement".

We will implement the function below to get the minimum amount of edits required given a source string and a target string. 

In [87]:
def min_edit_distance(source, target, ins_cost=1, del_cost=1, rep_cost=2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    m = len(source)
    n = len(target)
    # Initialize cost matrix with zeros and dimensions (m+1, n+1)
    D = np.zeros((m+1, n+1), dtype=int)

    ### START CODE HERE ###
    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1, m+1):
        D[row, 0] = row * del_cost

    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1, n+1):
        D[0, col] = col * ins_cost

    # Loop through row 1 to row m, both inclusive
    for row in range(1, m+1):
        # Loop through column 1 to column n, both inclusive
        for col in range(1, n+1):
            # Initialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost

            # Check to see if source character at the previous row
            # matches the target character at the previous column
            if source[row-1] == target[col-1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0

            # Update the cost at row, col based on previous entries in the cost matrix
            D[row, col] = min(
                D[row-1, col] + del_cost,       # Cost of deletion
                D[row, col-1] + ins_cost,       # Cost of insertion
                D[row-1, col-1] + r_cost        # Cost of replacement
            )

    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m, n]

    ### END CODE HERE ###
    return D, med

In [88]:
#DO NOT MODIFY THIS CELL
# testing your implementation 
source =  'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


In [89]:
#DO NOT MODIFY THIS CELL
# testing your implementation 
source =  'eer'
target = 'near'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  3 

   #  n  e  a  r
#  0  1  2  3  4
e  1  2  1  2  3
e  2  3  2  3  4
r  3  4  3  4  3


In [90]:
# Test your function
w1_unittest.test_min_edit_distance(min_edit_distance)

 All tests passed


We can now test several of our routines at once:

In [91]:
source = "eer"
targets = edit_one_letter(source,allow_switches = False)  #disable switches since min_edit_distance does not include them
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)  # set ins, del, sub costs all to one
    if min_edits != 1: print(source, t, min_edits)

In [92]:
source = "eer"
targets = edit_two_letters(source,allow_switches = False) #disable switches since min_edit_distance does not include them
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)  # set ins, del, sub costs all to one
    if min_edits != 2 and min_edits != 1: print(source, t, min_edits)

eer eer 0
